In [5]:
# Импорт библиотек для анализа данных
import pandas as pd
# import numpy as np

In [6]:
# Загрузка данных
data = pd.read_csv('data/titanic_full_cleared.csv', header=0)

In [7]:
# Настройки вывода
# Метод вывода для объекта pd.DataFrame
def _repr_html_styled_(self):
    styled_data = self.style.set_table_styles([
        {
            # Заголовки таблиц и индексы
            'selector': 'th', 
            'props': [
                ('font-family', 'Cascadia Code Light'),     # шрифт 
                ('font-weight', 'normal'),                  # толщина текста
                ('color', '#e5ce5c'),                       # цвет текста
                ('opacity', '1'),                           # прозрачность текста
                ('background-color', '#192d37'),            # цфет фона
                ('padding', '5px'),                         # отступы                
             ]
        },
        {
            # Стиль для индексов
            'selector': '.row_heading',
            'props': [
                ('color', '#828f9a'),
            ]
        },
        {
            # Содержимое ячеек
            'selector': 'td', 
            'props': [
                ('font-family', 'Cascadia Code Light'), 
                ('padding', '5px')
            ]
        },
        {
            # Четные строки содержимого таблицы
            'selector': 'tbody tr:nth-child(even)', 
            'props': [
                ('background-color', '#ffffff00')  # нет цвета ("прозрачные")
            ]
        },
        {
            # Цвет фона для нечетных строк (начиная со строки заголовка)
            'selector': 'tr:nth-of-type(odd)', 
            'props': [
                ('background-color', '#ffffff0a')
            ]
        },
        {
            # Стиль для индексов
            'selector': '.row_heading',
            'props': [
                ('color', '#828f9a'),
            ]
        },
        {
            # Hover эффект для ячейки
            'selector': 'td:hover',
            'props': [
                ('background-color', '#ffffff14'),
            ]
        },
          
    ])

    return styled_data._repr_html_() # возврат HTML-представления стилизованных данных


# Переопределение метода вывода объекта pd.DataFrame
pd.DataFrame._repr_html_ = _repr_html_styled_

In [8]:
# Полный набор данных
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,PersonsInTicket,Cost,FamilySize,IsAlone
0,0,3,0,2,1,0,1,1,1,1,7,2,0
1,1,1,1,3,1,0,4,2,4,2,2,2,0
2,1,3,1,2,0,0,1,1,2,1,8,1,1
3,1,1,1,3,1,0,4,1,4,2,1,2,0
4,0,3,0,3,0,0,1,1,1,1,8,1,1


In [ ]:
# Перемешиваем данные
# data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# Удаление признаков которые негативно отразились на результатах предсказания
X = data.drop(['SibSp', 'Parch', 'Fare', 'PersonsInTicket', 'FamilySize', 'IsAlone', 'Cost'], axis = 1)

In [ ]:
# Окончательный набор данных для обучения
X.head()

In [ ]:
# Разделяем данные на обучающий и тестовый наборы
train = X.iloc[:891].reset_index(drop=True)
test = X.iloc[891:].reset_index(drop=True)

In [ ]:
# Импорт библиотек для машинного обучения
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score

In [ ]:
# Сравнение классификаторов
classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()
]

# Создание DataFrame для хранения результатов работы моделей
log_cols = ['Classifier', 'Accuracy', 'Precision']
log = pd.DataFrame(columns=log_cols)

# Определение тренировочной выборки X_train и вектора целевых значений y_train
X_train = train.drop('Survived', axis=1)    # удаляем столбец Survived из тренировочных данных
y_train = train['Survived']                 # используем столбец Survived как вектор целевых значений

# Цикл перебора классификаторов и оценка их точности с использованием cross_val_score
for clf in classifiers:
    name = clf.__class__.__name__
    accuracy = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy').mean()
    precision = cross_val_score(clf, X_train, y_train, cv=10, scoring='precision').mean()
    log_entry = pd.DataFrame([[name, accuracy, precision]], columns=log_cols)
    log = pd.concat([log, log_entry], ignore_index=True)